In [1]:
# Licensed under the Apache License, Version 2.0 (the "License"); you may
# not use this file except in compliance with the License. You may obtain
# a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS, WITHOUT
# WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the
# License for the specific language governing permissions and limitations
# under the License.

In [1]:
!pip uninstall pixiedust -y

Uninstalling pixiedust-1.0.11:
Exception:
Traceback (most recent call last):
  File "/usr/local/src/bluemix_jupyter_bundle.v63/notebook/lib/python2.7/site-packages/pip/basecommand.py", line 215, in main
    status = self.run(options, args)
  File "/usr/local/src/bluemix_jupyter_bundle.v63/notebook/lib/python2.7/site-packages/pip/commands/uninstall.py", line 76, in run
    requirement_set.uninstall(auto_confirm=options.yes)
  File "/usr/local/src/bluemix_jupyter_bundle.v63/notebook/lib/python2.7/site-packages/pip/req/req_set.py", line 346, in uninstall
    req.uninstall(auto_confirm=auto_confirm)
  File "/usr/local/src/bluemix_jupyter_bundle.v63/notebook/lib/python2.7/site-packages/pip/req/req_install.py", line 754, in uninstall
    paths_to_remove.remove(auto_confirm)
  File "/usr/local/src/bluemix_jupyter_bundle.v63/notebook/lib/python2.7/site-packages/pip/req/req_uninstall.py", line 115, in remove
    renames(path, new_path)
  File "/usr/local/src/bluemix_jupyter_bundle.v63/notebook/

In [2]:
!pip install --upgrade pixiedust

Requirement already up-to-date: pixiedust in /usr/local/src/bluemix_jupyter_bundle.v63/notebook/lib/python2.7/site-packages
    100% |████████████████████████████████| 798kB 1.3MB/s eta 0:00:01
    100% |████████████████████████████████| 5.3MB 224kB/s eta 0:00:01
  Running setup.py bdist_wheel for mpld3 ... done
  Stored in directory: /gpfs/fs01/user/s046-815a748ee2714d-95beb86403e0/.cache/pip/wheels/69/bc/68/7ca3b696749d183e998968fc24b0ff3c5e119d9e68bf495b07
Successfully built mpld3


In [3]:
# This notebook requires version 1.0.6
import pixiedust

Pixiedust database opened successfully
Table VERSION_TRACKER created successfully


Pixiedust runtime updated. Please restart kernel
Table SPARK_PACKAGES created successfully
Table USER_PREFERENCES created successfully
Table service_connections created successfully


In [4]:


# With PixieDust, you can easily load CSV data from a URL into a PySpark DataFrame in the notebook.

accidents = pixiedust.sampleData("https://data.sfgov.org/Public-Safety/Police-Department-Incidents/tmnf-yvry/data")



Downloaded 455942 bytes
Creating pySpark DataFrame for 'https://data.sfgov.org/Public-Safety/Police-Department-Incidents/tmnf-yvry/data'. Please wait...
Loading file using 'SparkSession'
Successfully created pySpark DataFrame for 'https://data.sfgov.org/Public-Safety/Police-Department-Incidents/tmnf-yvry/data'


In [8]:
display(Police Department Incidents)


SyntaxError: invalid syntax (<ipython-input-8-7b001bfebbd2>, line 1)

In [9]:


# With PixieDust, you can easily load CSV data from a URL into a PySpark DataFrame in the notebook.

accidents = pixiedust.sampleData("https://data.sfgov.org/api/views/vv57-2fgy/rows.csv?accessType=DOWNLOAD")



Downloaded 226525 bytes
Creating pySpark DataFrame for 'https://data.sfgov.org/api/views/vv57-2fgy/rows.csv?accessType=DOWNLOAD'. Please wait...
Loading file using 'SparkSession'
Successfully created pySpark DataFrame for 'https://data.sfgov.org/api/views/vv57-2fgy/rows.csv?accessType=DOWNLOAD'


In [14]:
display(accidents)

In [12]:
from pyspark.sql.functions import udf
from pyspark.sql.types import *

# Get the hour value of a time string
# e.g. getHour("05:30") = 5
def getHour(s):
    return int(s.split(':')[0])

hr_udf = udf(getHour,IntegerType())

# Rename weekdays to enable mini time-series analysis
accidents = accidents.na.replace\
    (['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday'],\
    ['1-Mon','2-Tues','3-Wed','4-Thur','5-Fri','6-Sat','7-Sun'],\
    'DayOfWeek')

# Add Hour column and refine outcomes from traffic accidents
accidents = accidents.withColumn("Hour",hr_udf(accidents['Time']))\
    .withColumn("Res",\
    udf(lambda x: 'Arrest' if 'ARREST' in x else 'No Resolution' if x == 'NONE' else 'Other',StringType())\
    (accidents['Resolution']))

In [25]:
display(accidents)

In [27]:
accidents.registerTempTable("accidents")
taraval = sqlContext.sql("SELECT * FROM accidents WHERE PdDistrict='TARAVAL'")

In [28]:


display(taraval)

